# 자율주행을 위한 3구 가로 신호등 인식하기
  

**3구 가로 신호등 데이터 조건**
- class = 'traffic_light'  
- type = 'car'  
- direction = 'horizontal'  
- light_count = '3'

### 패키지 임포트

In [1]:
import json
import pandas as pd
import os
from PIL import Image
import matplotlib.pyplot as plt
import cv2

In [2]:
# value값으로 key값 찾기
def find_key(dict, val):
    keys = []
    for key, value in dict.items():
        if value == val:
            keys.append(key)
    return keys
    # return [key for key, value in dict.items() if value == val]
    # return next(key for key, value in dict.items() if value == val)

In [3]:
# color가 두개 이상 on인 경우나 on이 없을 경우 설정
def color_on(color_list):
    color_len = len(color_list)
    if color_len >= 2:
        for c in color_list:
            if c == 'red' or c == 'green' or c == 'yellow':
                pass
            else:
                return c
    elif color_len == 1:
        return color_list[0]
    elif color_len == 0:
        return 'light_off'

### 신호등 이미지 잘라서 저장하기

In [16]:
light = [] # 3구 신호등만 담을 리스트

filepath = "trainingData/training/train_daylight_8_label"    # json 경로
imagepath = "trainingData/training/train_daylight_8_image"    # 이미지 경로

# json 경로
classes = os.listdir(filepath)

# 색상 저장할 상위 폴더
dir_path = './data_color_8'
# dir_path 폴더 없으면 생성
if os.path.isdir(dir_path) == False:
    os.mkdir(dir_path)

    
for i, cls in enumerate(classes):
#     print("Openinig " + cls + "/")

    image_cnt = 0 # 이미지 저장할때 순번쓰려고 변수 선언
    
    data =  json.load(open(filepath + "/" + cls))
    df = pd.DataFrame(data["annotation"])
    
    # 이렇게 하나씩 조건문 걸어서 해줘야해요? 왜 컬럼이 없고 난리세요,, 뭐 다른 방법 없나요?
    if not df.empty:
        # 신호등만 (도로표지판 걸러)
        filter1 = (df['class'] == 'traffic_light')
        df = df[filter1]
        
        if not df.empty:
            # 자동차 신호등만 (보행자 신호등 걸러)
            filter2 = (df['type'] == 'car')
            df = df[filter2]
            
            if not df.empty:
                # 3구 신호등만
                filter3 = (df['light_count'] == '3')
                df = df[filter3]
                
                if not df.empty:
                    # 가로 신호등만 (세로 걸러)
                    filter4 = (df['direction'] == 'horizontal')
                    df = df[filter4]
                    
                    light.append(cls) # 필터링된 3구 가로 신호등 리스트에 담기
                    
                    # 이미지 가져오기
                    name = (os.path.splitext(cls))[0]    # json 확장자 빼기
                    image1 = name + ".jpg"
                    image = Image.open(imagepath + '/' + image1)
                    
                    
                    for j in range(len(df['box'])):
                        image_cnt += 1
                        box = df['box'][j:j+1]
                        
                        # 박스 좌표 가져오기
                        x = box.str.get(0)
                        y = box.str.get(1)
                        w = box.str.get(2)
                        h = box.str.get(3)
                        
                        # 이미지 영역 자르기
                        croppedImage = image.crop((x, y, w, h))
                        
                        for attr in df['attribute'][j:j+1].str.get(0):
                            try: 
                                color = find_key(attr, 'on')    # value값으로 key값 찾는 함수 호출
                            except: # 신호등에 불이 다 꺼져있을 경우
                                color = ['light_off']
                                
                        color = color_on(color)
                        if color == None:
                            color = 'light_off'
                        
                        # 신호등 색깔 폴더 경로
                        folder_path = dir_path + '/' + color

                        # 폴더 없으면 생성
                        if os.path.isdir(folder_path) == False:
                            os.mkdir(folder_path)

                        # 이미지 폴더에 저장
                        croppedImage.save(folder_path + '/' + name + "_" + str(image_cnt) + '.jpg', format = None)
                        
                    

In [8]:
#data = pd.read_json('./data/training/label_daylight/daylight/s01000200.json')
data = json.load(open('./trainingData/training/train_daylight_4_label/15391373.json'))
df = pd.DataFrame(data["annotation"])
df

,box,attribute,type,class,direction
0,"[1008, 506, 1024, 513]","[{'red': 'off', 'green': 'off', 'x_light': 'of...",car,traffic_light,horizontal
1,"[974, 507, 990, 512]","[{'red': 'off', 'green': 'off', 'x_light': 'of...",car,traffic_light,horizontal
